In [ ]:
%%html

<!DOCTYPE html>
<html>
    <body style="background-color: rgb(135, 132, 114);">
    <h1 style="color: cyan; font-size: 41px; font-family: 'Courier New'; background-color: rgb(135, 132, 114); border: 0px; text-align: left; padding: 0px 0px 0px 0px"><b><center>Dibujando con epiciclos y la transformada de Fourier</center></b></h1>
</html>

In [24]:
import numpy as np
import bqplot as bq
import ipywidgets as widgets
from bqplot import pyplot as bplt

file1 = open('modernlove.mp3', 'rb')
Audio1 = widgets.Audio(
    value    = file1.read(),
    autoplay = True,
    controls = True,
    layout   = widgets.Layout(justify_content='space-between', width='98.5%', height='20px'),
)
display(Audio1)

Button1 = widgets.Button(
          description  = 'Dibujar a mano',
          button_style = 'info',
          layout       = widgets.Layout(width = '200px', height = '30px', padding = '0px', margin = '0px', border = '0px')
)
Button2 = widgets.Button(
          description  = 'Dibujar con Fourier',
          button_style = 'info',
          layout       = widgets.Layout(width = '200px', height = '30px', padding = '0px', margin = '0px', border = '0px')
)
BoundedIntText1 = widgets.BoundedIntText(
          description  = 'Tiempo',
          min          = 1,
          step         = 1,
          value        = 25,
          layout       = widgets.Layout(width = '150px', height = '30px', padding = '0px 0px 0px 0px', margin = '0px 40px 0px 0px', border = '0px')
)
IntSlider1 = widgets.IntSlider(
          description  = 'Precision',
          min          = 1,
          step         = 1,
          value        = 1,
          layout       = widgets.Layout(width = '350px', height = '30px', padding = '0px 0px 0px 0px', margin = '0px', border = '0px')
)
IntProgress1 = widgets.IntProgress(
          description  = '',
          min          = 0,
          max          = 100,
          value        = 100,
          layout       = widgets.Layout(width = '250px', height = '36px', padding = '0px 0px 5px 0px', margin = '0px 30px 0px 0px', border = '0px')
)
HBox1 = widgets.HBox([Button1, IntSlider1, BoundedIntText1, IntProgress1, Button2],
          layout = widgets.Layout(width = '1285px', height = 'auto', justify_content = 'space-between')
)

x_size  = 500
y_size  = 500
scale_x = bq.LinearScale(min = -x_size, max = x_size)
scale_y = bq.LinearScale(min = -y_size, max = y_size)
ax_x    = bq.Axis(scale = scale_x, label = '', grid_lines = 'none', color='black')
ax_y    = bq.Axis(scale = scale_y, label = '', grid_lines = 'none', color='black', orientation = 'vertical')
m_fig   = dict(left=0, top=15, bottom=0, right=0)
fig3    = bplt.figure(marks = [], axes = [ax_x, ax_y], fig_margin = m_fig, layout=widgets.Layout(width='650px', height='650px'))
fig3.background_style = {'fill': 'Black'}
fig3.max_aspect_ratio = 1
fig3.min_aspect_ratio = 1

VBox1 = widgets.VBox([HBox1, fig3],
          layout = widgets.Layout(width = 'auto', height = 'auto', align_items = 'center')
)
display(VBox1)

Audio(value=b'ID3\x04\x00\x00\x00\x00\x01\x06TXXX\x00\x00\x00\x12\x00\x00\x03major_brand\x00isom\x00TXXX\x00\x…

In [3]:
def draw_hand(draw):
    
    lines5   = bq.Lines(x = [], y = [], scales={'x': scale_x, 'y': scale_y}, colors=['cyan'])
    scatter4 = bq.Scatter(x = [], y = [], scales = {'x': scale_x, 'y': scale_y}, default_size = 100, colors = ['white'], marker = 'circle', enable_move  = True, interactions = {'click': 'add'})

    def update_line(change = None):
        global X1
        global Y1
        lines5.x = np.append(lines5.x, scatter4.x[0])
        lines5.y = np.append(lines5.y, scatter4.y[0])
        X1       = lines5.x
        Y1       = lines5.y
        IntSlider1.max = int(len(X1) / 2 + 2)
    scatter4.observe(update_line, names=['x'])
    scatter4.observe(update_line, names=['y'])
    scatter4.update_on_move = True
    
    fig3  = bplt.figure(marks = [scatter4, lines5], axes = [ax_x, ax_y], fig_margin = m_fig, layout=widgets.Layout(width='650px', height='650px'))
    fig3.background_style = {'fill': 'Black'}
    fig3.max_aspect_ratio = 1
    fig3.min_aspect_ratio = 1
    a = list(VBox1.children); a.pop(1); VBox1.children = tuple(a); VBox1.children += (fig3,)
    
Button1.on_click(draw_hand) 

In [4]:
def dft(x):
    N = len(x)
    X = np.empty((N, 5))
    
    for k in range(N):
        re = 0
        im = 0
        for n in range(N):
            phi = (2 * np.pi * k * n) / N
            re  = re + x[n] * np.cos(phi)
            im  = im - x[n] * np.sin(phi)
        re = re / N
        im = im / N
        am = np.sqrt(re ** 2 + im ** 2)
        ph = np.arctan2(im, re)
        X[k,0] = re
        X[k,1] = im
        X[k,2] = k
        X[k,3] = am
        X[k,4] = ph
    return X

def draw_circles(draw):  
    t = BoundedIntText1.value
    IntProgress1.max = t
    tetha = np.linspace(0, 2 * np.pi, 25)
    X2 = X1[np.arange(0, len(X1), IntSlider1.value)]
    Y2 = Y1[np.arange(0, len(Y1), IntSlider1.value)]
    fourierX = dft(X2)
    fourierY = dft(Y2)
    lx = len(fourierX)
    ly = len(fourierY)
    
    Marks = []
    for i in range(2 * lx):
        Marks.append(bq.Lines(x = [], y = [], scales={'x': scale_x, 'y': scale_y}, colors=['white'], opacities = [.5], stroke_width = 1))
        Marks.append(bq.Lines(x = [], y = [], scales={'x': scale_x, 'y': scale_y}, colors=['cyan'],stroke_width = 1.5))
    
    Marks.append(bq.Lines(x = [], y = [], scales={'x': scale_x, 'y': scale_y}, colors=['red'], stroke_width = 1))    
    Marks.append(bq.Lines(x = [], y = [], scales={'x': scale_x, 'y': scale_y}, colors=['red'], stroke_width = 1))    
    Marks.append(bq.Lines(x = [], y = [], scales={'x': scale_x, 'y': scale_y}, colors=['yellow'], stroke_width = 2)) 
    
    fig3  = bplt.figure(marks = Marks, axes = [ax_x, ax_y], interaction = bq.interacts.PanZoom(scales={'x': [scale_x], 'y': [scale_y]}), fig_margin = m_fig, layout=widgets.Layout(width='650px', height='650px'))
    fig3.background_style = {'fill': 'Black'}
    fig3.max_aspect_ratio = 1
    fig3.min_aspect_ratio = 1
    a = list(VBox1.children); a.pop(1); VBox1.children = tuple(a); VBox1.children += (fig3,)

    i  = 0
    dt = (np.pi * 2) / ly
    for l in range(t):
   
        x = x_size / 2
        y = y_size / 2
        rot = 0
        for j in range(lx):
            X = x
            Y = y
            freq  = fourierX[j,2]
            r     = fourierX[j,3]
            phase = fourierX[j,4]
            x = x + r * np.cos(freq * i + phase + rot)
            y = y + r * np.sin(freq * i + phase + rot)
            fig3.marks[int(2 * j + 0)].x = r * np.cos(tetha) + X
            fig3.marks[int(2 * j + 0)].y = r * np.sin(tetha) + Y
            fig3.marks[int(2 * j + 1)].x = [X, x]
            fig3.marks[int(2 * j + 1)].y = [Y, y]
        fxpx = x
        fxpy = y
          
        x = x_size / -2
        y = y_size / -2
        rot = (np.pi / 2)
        for j in range(ly):
            X = x
            Y = y
            freq  = fourierY[j,2]
            r     = fourierY[j,3]
            phase = fourierY[j,4]
            x = x + r * np.cos(freq * i + phase + rot)
            y = y + r * np.sin(freq * i + phase + rot)
            fig3.marks[int(2 * j + (2 * ly))].x = r * np.cos(tetha) + X
            fig3.marks[int(2 * j + (2 * ly))].y = r * np.sin(tetha) + Y
            fig3.marks[int(2 * j + (2 * ly) + 1)].x = [X, x]
            fig3.marks[int(2 * j + (2 * ly) + 1)].y = [Y, y]
        fypx = x
        fypy = y 
        
        fig3.marks[4 * lx].x = [fxpx, fxpx]
        fig3.marks[4 * ly].y = [fxpy, fypy]
        fig3.marks[4 * lx + 1].x = [fypx, fxpx]
        fig3.marks[4 * ly + 1].y = [fypy, fypy]
        fig3.marks[4 * lx + 2].x = np.append(fig3.marks[4 * lx + 2].x, fxpx)
        fig3.marks[4 * ly + 2].y = np.append(fig3.marks[4 * ly + 2].y, fypy)
        i  = i + dt
        
        if i > 2 * np.pi:
            i = 0;
            fig3.marks[4 * lx + 2].x = []
            fig3.marks[4 * ly + 2].y = []
            
        IntProgress1.value = l + 1
            
Button2.on_click(draw_circles) 